In [1]:
!pip install kaggle
!mkdir -p ~/.kaggle


In [ ]:
%%writefile ~/.kaggle/kaggle.json
{
  "username": "rashvin20",
  "key": "REPLACE YOUR KEY HERE"
}


Writing /root/.kaggle/kaggle.json


In [3]:
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d moltean/fruits

Dataset URL: https://www.kaggle.com/datasets/moltean/fruits
License(s): CC-BY-SA-4.0
 99% 4.73G/4.78G [01:42<00:01, 27.3MB/s]
100% 4.78G/4.78G [01:42<00:00, 50.2MB/s]


In [5]:
!unzip fruits.zip -d fruits_dataset/

Streaming output truncated to the last 5000 lines.
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_21.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_213.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_217.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_221.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_225.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_229.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_233.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation/Quince 4/r0_237.jpg  
  inflating: fruits_dataset/fruits-360_original-size/fruits-36

In [6]:
!pip install -q wandb torchviz fvcore grad-cam scikit-learn graphviz
!apt-get -q install graphviz



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Reading package lists...
Building dependency tree...
Reading state information...
graphviz is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [7]:
import os, random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from PIL import Image

from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from fvcore.nn import FlopCountAnalysis
from torchviz import make_dot

import wandb

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)

os.makedirs("/content/outputs", exist_ok=True)


Running on: cuda


In [8]:
wandb.login()

wandb.init(
    project="vision-cw",
    name="cnn_augmented_A_dash",
    config={
        "dataset": "Fruits-360",
        "augmentation": True,
        "epochs": 30,
        "batch_size": 32,
        "lr": 1e-3
    }
)


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rashvin-rajanah-23 (rashvin-rajanah-23-ucl) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [9]:
train_path = "/content/fruits_dataset/fruits-360_original-size/fruits-360-original-size/Training"
val_path   = "/content/fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation"
transform = transforms.Compose([
    transforms.Resize((100,100)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor()
])

train_dataset_full = datasets.ImageFolder(train_path, transform=transform)
val_dataset_full   = datasets.ImageFolder(val_path, transform=transforms.Compose([
    transforms.Resize((100,100)),
    transforms.ToTensor()
]))

num_classes = len(train_dataset_full.classes)
print("Classes:", num_classes)
NUM_TRAIN = 10000
NUM_VAL   = 2000

train_idx = random.sample(range(len(train_dataset_full)), NUM_TRAIN)
val_idx   = random.sample(range(len(val_dataset_full)), NUM_VAL)

train_dataset = Subset(train_dataset_full, train_idx)
val_dataset   = Subset(val_dataset_full, val_idx)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)


Classes: 116


In [10]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 12 * 12, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.classifier(self.features(x))


model = CNN(num_classes).to(device)


In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

EPOCHS = 30
best_val_loss = float("inf")
patience, wait = 5, 0

train_losses, val_losses = [], []
train_accs, val_accs = [], []

for epoch in range(EPOCHS):
    model.train()
    correct, total, running_loss = 0, 0, 0

    for x, y in tqdm(train_loader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        correct += (out.argmax(1) == y).sum().item()
        total += y.size(0)

    train_loss = running_loss / len(train_loader)
    train_acc = correct / total

    model.eval()
    v_loss, v_correct, v_total = 0, 0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            v_loss += criterion(out, y).item()
            v_correct += (out.argmax(1) == y).sum().item()
            v_total += y.size(0)

    val_loss = v_loss / len(val_loader)
    val_acc = v_correct / v_total

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "val_loss": val_loss,
        "train_acc": train_acc,
        "val_acc": val_acc
    })

    print(f"Epoch {epoch+1}: TrainAcc={train_acc:.3f} ValAcc={val_acc:.3f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "/content/best_model.pth")
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            break
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(train_losses, label="Train")
plt.plot(val_losses, label="Val")
plt.title("Loss")
plt.legend()

plt.subplot(1,2,2)
plt.plot(train_accs, label="Train")
plt.plot(val_accs, label="Val")
plt.title("Accuracy")
plt.legend()

plt.tight_layout()
plt.savefig("/content/outputs/training_curves.jpg", dpi=300)
plt.close()

wandb.log({"training_curves": wandb.Image("/content/outputs/training_curves.jpg")})
model.load_state_dict(torch.load("/content/best_model.pth", map_location=device))
model.eval()

y_true, y_pred = [], []

with torch.no_grad():
    for x, y in val_loader:
        x, y = x.to(device), y.to(device)
        preds = model(x).argmax(1)
        y_true.extend(y.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

f1 = f1_score(y_true, y_pred, average="macro")
print("Macro F1:", f1)
wandb.log({"macro_f1": f1})

cm = confusion_matrix(y_true, y_pred, normalize="true")
disp = ConfusionMatrixDisplay(cm)

disp.plot(cmap="Blues")
plt.savefig("/content/outputs/confusion_matrix.jpg", dpi=300)
plt.close()

wandb.log({"confusion_matrix": wandb.Image("/content/outputs/confusion_matrix.jpg")})

img_path = val_dataset_full.imgs[0][0]
img = Image.open(img_path).resize((100,100)).convert("RGB")
img_np = np.array(img) / 255.0
tensor = transforms.ToTensor()(img).unsqueeze(0).to(device)

target_layer = model.features[-2]
cam = GradCAM(model=model, target_layers=[target_layer])
grayscale_cam = cam(input_tensor=tensor)[0]

vis = show_cam_on_image(img_np, grayscale_cam, use_rgb=True)
plt.imshow(vis); plt.axis("off")
plt.savefig("/content/outputs/gradcam.jpg", dpi=300)
plt.close()

wandb.log({"gradcam": wandb.Image("/content/outputs/gradcam.jpg")})

dummy = torch.randn(1, 3, 100, 100).to(device)

flops = FlopCountAnalysis(model, dummy)
wandb.log({
    "FLOPs": flops.total(),
    "Parameters": sum(p.numel() for p in model.parameters())
})

dot = make_dot(model(dummy), params=dict(model.named_parameters()))
dot.format = "png"
dot.render("/content/outputs/cnn_architecture")

wandb.log({"architecture": wandb.Image("/content/outputs/cnn_architecture.png")})
wandb.finish()



100%|██████████| 313/313 [01:01<00:00,  5.12it/s]


Epoch 1: TrainAcc=0.035 ValAcc=0.127


100%|██████████| 313/313 [00:47<00:00,  6.60it/s]


Epoch 2: TrainAcc=0.069 ValAcc=0.142


100%|██████████| 313/313 [00:46<00:00,  6.76it/s]


...

100%|██████████| 313/313 [00:46<00:00,  6.69it/s]


Epoch 29: TrainAcc=0.284 ValAcc=0.767


100%|██████████| 313/313 [00:46<00:00,  6.80it/s]


Epoch 30: TrainAcc=0.301 ValAcc=0.766
Macro F1: 0.660782498587841


FLOPs,▁
Parameters,▁
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
macro_f1,▁
train_acc,▁▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇██
train_loss,█▆▆▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
val_acc,▁▁▂▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▇▆▇▇▇▇▇████
val_loss,█▇▇▅▅▅▅▄▄▄▄▄▄▃▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
FLOPs,106668288
Parameters,4842356
epoch,30


In [7]:

import os, random
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import wandb


VAL_PATH = "/content/fruits_dataset/fruits-360_original-size/fruits-360-original-size/Validation"
WEIGHTS_PATH = "/content/best_model.pth"
NUM_VAL_TEST = 3000
BATCH_SIZE = 32

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)

os.makedirs("/content/outputs", exist_ok=True)


class CNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 12 * 12, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.classifier(self.features(x))

tmp_ds = datasets.ImageFolder(
    VAL_PATH,
    transform=transforms.ToTensor()
)
num_classes = len(tmp_ds.classes)

print(f"Number of classes: {num_classes}")


model = CNN(num_classes).to(device)
model.load_state_dict(torch.load(WEIGHTS_PATH, map_location=device))
model.eval()

print(f"✅ Loaded trained weights from {WEIGHTS_PATH}")


clean_transform = transforms.Compose([
    transforms.Resize((100,100)),
    transforms.ToTensor()
])

brightness_transform = transforms.Compose([
    transforms.Resize((100,100)),
    transforms.ColorJitter(brightness=0.5),
    transforms.ToTensor()
])

occlusion_transform = transforms.Compose([
    transforms.Resize((100,100)),
    transforms.ToTensor(),
    transforms.RandomErasing(p=1.0, scale=(0.1, 0.2))
])


base_ds = datasets.ImageFolder(VAL_PATH, transform=clean_transform)

# Sample same indices for all conditions
indices = random.sample(range(len(base_ds)), NUM_VAL_TEST)

clean_ds  = Subset(datasets.ImageFolder(VAL_PATH, transform=clean_transform), indices)
bright_ds = Subset(datasets.ImageFolder(VAL_PATH, transform=brightness_transform), indices)
occl_ds   = Subset(datasets.ImageFolder(VAL_PATH, transform=occlusion_transform), indices)

print(f"Validation images tested (robustness): {len(clean_ds)}")

clean_loader  = DataLoader(clean_ds,  batch_size=BATCH_SIZE, shuffle=False)
bright_loader = DataLoader(bright_ds, batch_size=BATCH_SIZE, shuffle=False)
occl_loader   = DataLoader(occl_ds,   batch_size=BATCH_SIZE, shuffle=False)


def eval_accuracy(loader):
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            preds = model(x).argmax(1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    return correct / total


acc_clean  = eval_accuracy(clean_loader)
acc_bright = eval_accuracy(bright_loader)
acc_occl   = eval_accuracy(occl_loader)

drop_bright = acc_clean - acc_bright
drop_occl   = acc_clean - acc_occl

print("\n📊 ROBUSTNESS RESULTS")
print(f"Clean accuracy:        {acc_clean:.4f}")
print(f"Brightness accuracy:   {acc_bright:.4f}")
print(f"Occlusion accuracy:    {acc_occl:.4f}")
print(f"Brightness drop:       {drop_bright:.4f}")
print(f"Occlusion drop:        {drop_occl:.4f}")


labels = ["Clean", "Brightness", "Occlusion"]
accuracies = [acc_clean, acc_bright, acc_occl]

plt.figure(figsize=(6,4))
plt.bar(labels, accuracies)
plt.ylim(0, 1.0)
plt.ylabel("Accuracy")
plt.title("Robustness Evaluation (Accuracy)")
plt.tight_layout()
plt.savefig("/content/outputs/robustness_accuracy.jpg", dpi=300)
plt.close()


plt.figure(figsize=(6,4))
plt.bar(["Brightness Drop", "Occlusion Drop"], [drop_bright, drop_occl])
plt.ylabel("Accuracy Drop")
plt.title("Robustness Degradation")
plt.tight_layout()
plt.savefig("/content/outputs/robustness_drop.jpg", dpi=300)
plt.close()

print("📁 Saved robustness plots in /content/outputs/")


try:
    wandb.log({
        "robust_acc_clean": acc_clean,
        "robust_acc_brightness": acc_bright,
        "robust_acc_occlusion": acc_occl,
        "brightness_drop": drop_bright,
        "occlusion_drop": drop_occl,
        "robustness_accuracy_plot": wandb.Image("/content/outputs/robustness_accuracy.jpg"),
        "robustness_drop_plot": wandb.Image("/content/outputs/robustness_drop.jpg")
    })
    print("📈 Robustness metrics logged to W&B")
except:
    print("ℹ️ W&B not active — skipped logging")


Running on: cuda
Number of classes: 116
✅ Loaded trained weights from /content/best_model.pth
Validation images tested (robustness): 3000

📊 ROBUSTNESS RESULTS
Clean accuracy:        0.7703
Brightness accuracy:   0.6543
Occlusion accuracy:    0.5667
Brightness drop:       0.1160
Occlusion drop:        0.2037
📁 Saved robustness plots in /content/outputs/
ℹ️ W&B not active — skipped logging
